In [ ]:
# India_flood_testing - DEM extent authoritative, no AOI, urban-only LULC==13
# Single-file Jupyter / Python script
# Dependencies (run these once):
# !pip install rasterio geopandas shapely fiona pyproj
# !pip install scikit-learn xgboost joblib
# !pip install pandas numpy matplotlib scipy

import os, warnings, math, sys, time, zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rasterio
from rasterio import windows as rw
from rasterio.enums import Resampling
from rasterio.warp import reproject, Resampling as WarpResampling
import joblib
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay
from sklearn.ensemble import RandomForestClassifier
from scipy import ndimage
import geopandas as gpd
from pyproj import Transformer

warnings.filterwarnings("ignore")

# ---------- small helper for OneHotEncoder ----------
def make_onehot(handle_unknown="ignore"):
    try:
        return OneHotEncoder(handle_unknown=handle_unknown, sparse_output=False)
    except TypeError:
        return OneHotEncoder(handle_unknown=handle_unknown, sparse=False)

# ---------- USER EDITS ----------
base_dir   = r"C:\Users\DELL\Desktop\Cognitud\Urban Flood"
output_dir = os.path.join(base_dir, "outputs_cut_n30w120")
aligned_dir= os.path.join(base_dir, "aligned_cut_n30w120")
os.makedirs(output_dir, exist_ok=True)
os.makedirs(aligned_dir, exist_ok=True)

# Predictor rasters (edit to your files)
factor_paths = {
    "Rain":      os.path.join(base_dir, "rainfall_intensity_ssp_1.tif"),
    "Elevation": os.path.join(base_dir, "cut_n30w120.tif"),  # DEM (authoritative)
    "Landcover": os.path.join(base_dir, "global_SSP1_RCP26_2030.tif")
}
categorical_factors = ["Landcover"]

# Training CSV points
points_csv = os.path.join(base_dir, "nonflood_points_no_buffer.csv")
LAT_COL = "LATITUDE"
LON_COL = "LONGITUDE"
LABEL_COL_PREFERRED = "id"

# Model files
rf_model_path  = os.path.join(output_dir, "rf_pipe.pkl")
xgb_model_path = os.path.join(output_dir, "xgb_pipe.pkl")

# Tiling & memory knobs
OVERLAP = 32
ROWS_PER_CHUNK = 1024
RANDOM_STATE = 42
ALIGN_INPUTS_ONCE = True
AUTO_SKIP_TRAIN_IF_MODELS_EXIST = True
USE_GPU = False
BLOCK_SIZE = 1024
DO_HYPERPARAM_TUNE = False

# GDAL/Rasterio runtime options
rasterio_env = dict(GDAL_CACHEMAX=512, NUM_THREADS="ALL_CPUS")

# ---------- check inputs exist ----------
for k, v in factor_paths.items():
    if not os.path.exists(v):
        raise FileNotFoundError(f"Missing predictor: {v}")
if not os.path.exists(points_csv):
    raise FileNotFoundError(f"Missing training CSV: {points_csv}")

# ---------- helper: ensure DEM has CRS (assign EPSG:4326 if missing) ----------
def ensure_crs_assigned(src_path, crs_str="EPSG:4326", out_dir=None):
    """
    If raster has no CRS, write a copy with the provided CRS and return the path to the file
    If raster already has CRS, return original path.
    """
    with rasterio.Env(**rasterio_env):
        with rasterio.open(src_path) as src:
            if src.crs is not None:
                return src_path
            # read full array and profile
            data = src.read(1)
            profile = src.profile.copy()
            profile.update(crs=crs_str)

    # prepare output path
    basename = os.path.basename(src_path)
    name, ext = os.path.splitext(basename)
    out_name = f"{name}_proj{ext}"
    if out_dir is None:
        out_dir = os.path.dirname(src_path)
    out_path = os.path.join(out_dir, out_name)

    # write new file with assigned CRS
    with rasterio.Env(**rasterio_env):
        with rasterio.open(out_path, "w", **profile) as dst:
            dst.write(data, 1)
    print(f"Assigned CRS {crs_str} to '{src_path}' -> wrote: {out_path}")
    return out_path

# ---------- sampling & alignment helpers ----------
def sample_points_from_raster(src_path, lons, lats, assumed_src_crs="EPSG:4326"):
    with rasterio.open(src_path) as src:
        if src.crs is None:
            xs, ys = lons, lats
        elif src.crs.to_string() != "EPSG:4326":
            tfm = Transformer.from_crs("EPSG:4326", src.crs, always_xy=True)
            xs, ys = tfm.transform(lons, lats)
        else:
            xs, ys = lons, lats
        samples = list(src.sample(zip(xs, ys)))
        arr = np.array([s[0] if len(s) else np.nan for s in samples], dtype="float32")
        if src.nodata is not None:
            arr[arr == src.nodata] = np.nan
        return arr

def align_to_ref(src_path, ref_profile, resampling, dst_path, assumed_src_crs="EPSG:4326"):
    ref_crs = ref_profile["crs"]
    ref_transform = ref_profile["transform"]
    out_h, out_w = ref_profile["height"], ref_profile["width"]
    with rasterio.Env(**rasterio_env):
        with rasterio.open(src_path) as src:
            src_arr = src.read(1).astype(np.float32)
            if src.nodata is not None:
                src_arr[src_arr == src.nodata] = np.nan
            dst = np.full((out_h, out_w), np.float32(np.nan), dtype=np.float32)
            reproject(
                source=src_arr,
                destination=dst,
                src_transform=src.transform,
                src_crs=src.crs or assumed_src_crs,
                dst_transform=ref_transform,
                dst_crs=ref_crs,
                resampling=resampling,
                src_nodata=np.nan,
                dst_nodata=np.float32(np.nan),
                init_dest_nodata=True,
            )
    prof = ref_profile.copy()
    prof.update(count=1, dtype="float32", nodata=np.float32(np.nan),
                compress="lzw", tiled=True, blockxsize=BLOCK_SIZE, blockysize=BLOCK_SIZE, predictor=2)
    with rasterio.open(dst_path, "w", **prof) as dst_src:
        dst_src.write(dst, 1)

# small array utilities
def expanded_window(win, max_w, max_h, overlap):
    col0 = max(0, int(win.col_off) - overlap)
    row0 = max(0, int(win.row_off) - overlap)
    col1 = min(max_w, int(win.col_off + win.width) + overlap)
    row1 = min(max_h, int(win.row_off + win.height) + overlap)
    return rw.Window(col_off=col0, row_off=row0, width=col1 - col0, height=row1 - row0)


def crop_array_to_window(arr, big_win, target_win):
    r0 = int(target_win.row_off) - int(big_win.row_off)
    c0 = int(target_win.col_off) - int(big_win.col_off)
    r1 = r0 + int(target_win.height)
    c1 = c0 + int(target_win.width)
    return arr[r0:r1, c0:c1]


def fill_nan_nearest(a):
    if not np.isnan(a).any():
        return a
    mask = np.isnan(a)
    _, inds = ndimage.distance_transform_edt(mask, return_indices=True)
    filled = a[tuple(inds)]
    return filled


def predict_window(models, feat_dict, out_shape, feature_cols):
    h, w = out_shape
    n = h * w
    flat_feats = [feat_dict[c].reshape(n) for c in feature_cols]
    stacked = np.column_stack(flat_feats)
    mask = np.all(~np.isnan(stacked), axis=1)
    proba_mean = np.full(n, np.nan, dtype=np.float32)
    chunk_size = ROWS_PER_CHUNK * w
    for start in range(0, n, chunk_size):
        end = min(n, start + chunk_size)
        submask = mask[start:end]
        if not np.any(submask):
            continue
        df = pd.DataFrame({col: stacked[start:end, i][submask] for i, col in enumerate(feature_cols)})
        for c in categorical_factors:
            if c in df.columns:
                df[c] = df[c].astype(object)
        probs = []
        for pipe in models:
            probs.append(pipe.predict_proba(df)[:, 1])
        mean_probs = np.nanmean(np.vstack(probs), axis=0)
        proba_mean[start:end][submask] = mean_probs
    return proba_mean.reshape((h, w))

# ---------- PART A: TRAINING ----------
print("=== PART A: TRAINING MODELS ===")
feature_cols = list(factor_paths.keys())
if "Elevation" not in factor_paths:
    raise ValueError("You must provide 'Elevation' (DEM) in factor_paths to establish prediction extent/CRS.")

models_exist = os.path.exists(rf_model_path) and os.path.exists(xgb_model_path)
if AUTO_SKIP_TRAIN_IF_MODELS_EXIST and models_exist:
    print("Models found. Loading and skipping training.")
    rf_pipe  = joblib.load(rf_model_path)
    xgb_pipe = joblib.load(xgb_model_path)
else:
    points = pd.read_csv(points_csv)
    for col_needed in [LAT_COL, LON_COL]:
        if col_needed not in points.columns:
            raise ValueError(f"CSV missing required column: {col_needed}")
    label_col = LABEL_COL_PREFERRED if LABEL_COL_PREFERRED in points.columns else "id"

    lats   = points[LAT_COL].values
    lons   = points[LON_COL].values
    labels = points[label_col].astype(int).values

    samples = {}
    for name, path in factor_paths.items():
        print(f"Sampling: {name}")
        arr = sample_points_from_raster(path, lons, lats)
        if name in categorical_factors:
            arr_int = np.where(np.isnan(arr), np.nan, np.round(arr).astype(np.float32))
            samples[name] = arr_int
        else:
            samples[name] = arr

    df_train = pd.DataFrame(samples)
    df_train["Label"] = labels
    X = df_train[feature_cols]
    y = df_train["Label"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=RANDOM_STATE)

    num_cols = [c for c in feature_cols if c not in categorical_factors]
    cat_cols = [c for c in feature_cols if c in categorical_factors]

    num_pipe = Pipeline([("imputer", KNNImputer(n_neighbors=5)), ("scaler", StandardScaler()),])
    cat_pipe = Pipeline([("imputer", SimpleImputer(strategy="most_frequent")), ("onehot", make_onehot(handle_unknown="ignore")),])
    transformers = []
    if num_cols: transformers.append(("num", num_pipe, num_cols))
    if cat_cols: transformers.append(("cat", cat_pipe, cat_cols))
    preprocessor = ColumnTransformer(transformers, remainder="drop")

    rf = RandomForestClassifier(n_estimators=400, max_depth=16, n_jobs=-1, class_weight="balanced_subsample", random_state=RANDOM_STATE)
    tree_method = "gpu_hist" if USE_GPU else "hist"
    predictor = "gpu_predictor" if USE_GPU else None
    xgbc = xgb.XGBClassifier(n_estimators=400, learning_rate=0.05, max_depth=8, subsample=0.8,
                             colsample_bytree=0.6, objective="binary:logistic",
                             scale_pos_weight=max(1.0, (y_train == 0).sum() / max(1, (y_train == 1).sum())),
                             tree_method=tree_method, predictor=predictor, random_state=RANDOM_STATE,
                             use_label_encoder=False, eval_metric="logloss",)

    rf_pipe  = Pipeline([("pre", preprocessor), ("clf", rf)])
    xgb_pipe = Pipeline([("pre", preprocessor), ("clf", xgbc)])

    if DO_HYPERPARAM_TUNE:
        rf_param = {"clf__n_estimators": [100, 200, 300], "clf__max_depth": [8, 12, 16], "clf__max_features": ["sqrt", 0.5, None],}
        xgb_param = {"clf__n_estimators": [200, 400], "clf__max_depth": [4, 6, 8], "clf__learning_rate": [0.01, 0.05, 0.1], "clf__subsample": [0.6, 0.8, 1.0],}
        rf_search = RandomizedSearchCV(rf_pipe, rf_param, n_iter=6, scoring="roc_auc", n_jobs=-1, cv=3, random_state=RANDOM_STATE)
        xgb_search = RandomizedSearchCV(xgb_pipe, xgb_param, n_iter=6, scoring="roc_auc", n_jobs=-1, cv=3, random_state=RANDOM_STATE)
        rf_search.fit(X_train, y_train)
        xgb_search.fit(X_train, y_train)
        rf_pipe = rf_search.best_estimator_
        xgb_pipe = xgb_search.best_estimator_
    else:
        for c in cat_cols:
            X_train[c] = X_train[c].astype(object)
            X_test[c]  = X_test[c].astype(object)
        rf_pipe.fit(X_train, y_train)
        xgb_pipe.fit(X_train, y_train)

    # Quick eval & save
    plt.figure(figsize=(6, 5))
    for name, pipe in {"RF": rf_pipe, "XGB": xgb_pipe}.items():
        y_pred  = pipe.predict(X_test)
        y_proba = pipe.predict_proba(X_test)[:, 1]
        acc = accuracy_score(y_test, y_pred)
        auc = roc_auc_score(y_test, y_proba)
        print(f"\n{name} | Accuracy={acc:.3f}, AUC={auc:.3f}")
        print(confusion_matrix(y_test, y_pred))
        print(classification_report(y_test, y_pred, digits=3))
        RocCurveDisplay.from_predictions(y_test, y_proba, name=name)
    plt.title("ROC Curves — RF vs XGB")
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "roc_rf_xgb.png"), dpi=150)
    plt.close()

    joblib.dump(rf_pipe,  rf_model_path,  compress=3)
    joblib.dump(xgb_pipe, xgb_model_path, compress=3)
    print("Models saved:", rf_model_path, xgb_model_path)

# ---------- PART B: ALIGN INPUTS (DEM reference) ----------
print("\n=== PART B: ALIGNING INPUTS TO DEM ===")

# Ensure DEM has CRS assigned (assign WGS84 if missing)
dem_path = factor_paths["Elevation"]
dem_path = ensure_crs_assigned(dem_path, crs_str="EPSG:4326", out_dir=aligned_dir)
# if a new file was written, update factor_paths reference for elevation
factor_paths["Elevation"] = dem_path

with rasterio.open(dem_path) as dem_ds:
    ref_profile = dem_ds.profile.copy()
    ref_crs = dem_ds.crs
    ref_transform = dem_ds.transform
    dem_width, dem_height = dem_ds.width, dem_ds.height
    dem_bounds = dem_ds.bounds
    print("DEM CRS:", ref_crs, "size:", dem_width, dem_height)
    print("DEM bounds:", dem_bounds)

if ALIGN_INPUTS_ONCE:
    for name in list(factor_paths.keys()):
        src = factor_paths[name]
        dst = os.path.join(aligned_dir, f"{name}.tif")
        if not os.path.exists(dst):
            resamp = WarpResampling.nearest if name in categorical_factors else WarpResampling.bilinear
            print(f"Aligning {name} -> {dst}  (resampling: {'nearest' if name in categorical_factors else 'bilinear'})")
            align_to_ref(src, ref_profile, resamp, dst, assumed_src_crs="EPSG:4326")
        factor_paths[name] = dst

# ---------- PART C: STREAMED PREDICTION (full DEM extent, URBAN-only where LULC == 6) ----------
print("\n=== PART C: STREAMED PREDICTION (DEM extent, URBAN only LULC==13) ===")
rf_pipe = joblib.load(rf_model_path)
xgb_pipe = joblib.load(xgb_model_path)
models = [rf_pipe, xgb_pipe]

final_profile = ref_profile.copy()
final_profile.update(driver="GTiff", dtype="float32", count=1, nodata=np.float32(np.nan),
                     compress="lzw", BIGTIFF="YES", tiled=True, blockxsize=BLOCK_SIZE, blockysize=BLOCK_SIZE, predictor=2)
final_map = os.path.join(output_dir, "UB_Flood_cut_n30w120.tif")  # change the final output name --------------------------------------------->

URBAN_VALUE = 6  # Landcover value indicating urban

with rasterio.Env(**rasterio_env):
    with rasterio.open(factor_paths["Elevation"]) as ref_ds, rasterio.open(final_map, "w", **final_profile) as out_ds:
        width, height = ref_ds.width, ref_ds.height

        # open aligned predictors (these are the aligned files in aligned_dir)
        predictors = {name: rasterio.open(path) for name, path in factor_paths.items()}
        feature_cols = list(predictors.keys())

        total_tiles = math.ceil(height / BLOCK_SIZE) * math.ceil(width / BLOCK_SIZE)
        processed = 0
        t0 = time.time()

        for r_off in range(0, height, BLOCK_SIZE):
            h = min(BLOCK_SIZE, height - r_off)
            for c_off in range(0, width, BLOCK_SIZE):
                w = min(BLOCK_SIZE, width - c_off)
                tile_win = rw.Window(col_off=c_off, row_off=r_off, width=w, height=h)

                big_win = expanded_window(tile_win, width, height, OVERLAP)

                feat_dict = {}
                all_nan = True
                urban_mask = np.zeros((h, w), dtype=bool)

                # Read each predictor. For Landcover we keep a raw copy (no nearest fill)
                for name, src in predictors.items():
                    arr_big = src.read(1, window=big_win).astype(np.float32)
                    if src.nodata is not None:
                        arr_big[arr_big == src.nodata] = np.nan
                    tile_arr = crop_array_to_window(arr_big, big_win, tile_win)

                    if name == "Landcover":
                        # Keep a raw copy to determine urban mask (do NOT fill)
                        tile_arr_raw = tile_arr.copy()
                        # compute urban mask — compare with tolerance in case of float storage
                        # round before comparison if values should be integers
                        with np.errstate(invalid="ignore"):
                            urban_mask = np.isfinite(tile_arr_raw) & (np.round(tile_arr_raw) == URBAN_VALUE)

                        # For model features, fill missing values so ML pipeline can run
                        tile_arr_filled = fill_nan_nearest(tile_arr)
                        feat_dict[name] = tile_arr_filled
                        if all_nan and np.any(~np.isnan(tile_arr_filled)):
                            all_nan = False
                    else:
                        # For numeric predictors fill NaNs so the model has inputs
                        tile_arr_filled = tile_arr
                        # if entirely NaN, attempt nearest fill (keeps continuity across tiles)
                        tile_arr_filled = fill_nan_nearest(tile_arr_filled)
                        feat_dict[name] = tile_arr_filled
                        if all_nan and np.any(~np.isnan(tile_arr_filled)):
                            all_nan = False

                # If there are no urban pixels in this tile, skip predicting and write NaNs
                if not urban_mask.any() or all_nan:
                    out_ds.write(np.full((h, w), np.float32(np.nan)), 1, window=tile_win)
                else:
                    # predict (this returns probabilities for entire tile)
                    proba = predict_window(models, feat_dict, (h, w), feature_cols).astype(np.float32)

                    # Ensure predictions only kept inside urban mask
                    proba[~urban_mask] = np.nan

                    # Also ensure any places with invalid features are NaN
                    out_ds.write(proba, 1, window=tile_win)

                processed += 1
                if processed % 200 == 0 or processed == total_tiles:
                    pct = 100.0 * processed / max(1, total_tiles)
                    elapsed = (time.time() - t0) / 60.0
                    print(f"\rProcessed {processed}/{total_tiles} tiles ({pct:.1f}%) in {elapsed:.1f} min", end="")

        print("\nAll tiles processed.")
        for src in predictors.values():
            src.close()

print("\n✅ Final urban-only map saved:", final_map)

# ---------- Quicklook (urban-only) ----------
print("\nSaving quicklook PNG (urban-only)...")
with rasterio.open(final_map) as ds:
    out_h = max(1, ds.height // 8)
    out_w = max(1, ds.width  // 8)
    small = ds.read(1, out_shape=(out_h, out_w), resampling=Resampling.average)

plt.figure(figsize=(12, 6))
plt.imshow(small, cmap="viridis")
plt.colorbar(label="Susceptibility (0-1)")
plt.title("Ensemble Susceptibility (URBAN only, LULC==13)")
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "flood_quicklook_urban_only.png"), dpi=150)
plt.close()
plt.show()
print("Quicklook saved to", os.path.join(output_dir, "flood_quicklook_urban_only.png"))


=== PART A: TRAINING MODELS ===
Sampling: Rain
